# Marginal likelihood for hierarchical model

|Study          | Treatment group |   Control group  |
|---------------|-----------------|------------------|
|Di Rienzo 2014 |     20 / 23     |        9 / 15    |
|Galli 1994     |     10 / 16     |       11 / 18    |
|Kaufman 1974   |     13 / 16     |        4 / 10    |
|Qin 2014       |     35 / 45     |       21 / 39    |
|Sanchez 2012   |     22 / 31     |       12 / 29    |
|Silny 2006     |      7 / 10     |        0 / 10    |
|**Totals**     |    107 / 141    |       57 / 121   |

**Model 2:** According to the hierarchical model with binomial likelihood, beta priors, and Gamma(2, 0.5) hyperpriors, the marginal likelihood is

$$ P(\text{data}) = \left[\prod_{i=1}^6 (s_i+f_i+1)\,\text{B}(s_i+1,f_i+1) \right]^{-1} \int_0^{\infty}\int_0^{\infty} P(\alpha,\beta)\, \text{B}(\alpha,\beta)^{-6} \prod_{i=1}^6 \text{B}(\alpha+s_i, \beta+f_i)\ \text{d}\beta\,\text{d}\alpha$$

where

$$ P(\alpha,\beta) = \text{Gamma}(\alpha\,|\,2,0.5)\,\text{Gamma}(\beta\,|\,2,0.5) $$

## Task 1
Read through the code below and discuss how these functions compute the marginal likelihood of the hierarchical model.

In [1]:
import numpy as np
import scipy.stats as sts
from scipy import special
from scipy.integrate import nquad


# The integrand for the double integral over a (alpha) and b (beta).
def integrand(a, b, data):
    n = data.shape[1]  # Number of data
    integrand = (
        sts.gamma(a=2, scale=1/0.5).pdf(a) *
        sts.gamma(a=2, scale=1/0.5).pdf(b) *
        special.beta(a, b) ** -n *
        np.product(special.beta(a + data[0,:], b + data[1,:])))
    return integrand


def hierarchical_marginal_likelihood(data):
    # Compute the constant outside the double integral
    constant = np.product((data[0,:] + data[1,:] + 1) * special.beta(data[0,:] + 1, data[1,:] + 1))

    # Compute the double integral.
    # NOTE: After much experimentation, the upper bounds of dblquad are set
    # to 50 rather than inf since the integral just diverges if you try inf.
    # It shouldn't, but it does.
    integral, error = nquad(integrand, ranges=[(0, 50), (0, 50)], args=[data])

    return {
        'marginal_likelihood': integral / constant,  # Value of the integral
        'error': error / constant}  # Numerical error in integral

In [2]:
data_treatment = np.array([[20, 10, 13, 35, 22, 7], [3, 6, 3, 10, 9, 3]])
data_control = np.array([[9, 11, 4, 21, 12, 0], [6, 7, 6, 18, 17, 10]])

## Task 2
Compute the marginal likelihood for the hierarchical model for each of the treatment and control group data sets.

In [5]:
print("Treatment", hierarchical_marginal_likelihood(data_treatment))
print("Control", hierarchical_marginal_likelihood(data_control))

Treatment {'marginal_likelihood': 1.4579106203822117e-07, 'error': 8.684205211169477e-08}


Control {'marginal_likelihood': 2.4349786578049566e-08, 'error': 8.494153804343146e-09}


## Task 3

From the previous activity, the marginal likelihoods for the non-hierarchical model were:
* Treatment group data: $8.234592 \times 10^{-7}$
* Control group data: $5.636806 \times 10^{-9}$

Compare the marginal likelihoods for the non-hierarchical and hierarchical models on each of the treatment and control group data sets.
1. Is the non-hierarchical or the hierarchical model better for this data set?
2. Explain why the one model is better than the other.

In [7]:
model1_treat = 8.234592e-7
model1_control = 5.636806e-9
model2_treat = 1.4579106203822117e-07
model2_control = 2.4349786578049566e-08

Hierarchical

* Treatment group data: $1.4579 \times 10^{-7}$
* Control group data: $2.43497 \times 10^{-8}$

In [8]:
model1_treat/model2_treat

5.648214564649503

In [9]:
model1_control/model2_control

0.23149303514148137

In [11]:
model1_treat*model1_control/(model2_treat*model2_control)

1.3075223327010344

For the treatment data, the first model is more likely, while for the control data, the second model is more likely.